### Hungarian Method 
#### For Assignment Problem
#### Model the problem as a weighted bipartite matching one
##### Youn-Long Lin, DEpartment of Computer Science, National Tsing Hua University, Taiwan

In [201]:
'''This cell is for max network flow problem
   It is used for 
      1. Maximum cardinality bipartite matching
      2. Constructing Minimum vertex cover of a bipartite graph from above found matching
'''
import numpy as np

class Edge(object):
    def __init__(self, u, v, w, f):
        self.source = u
        self.sink = v
        self.capacity = w
        self.flow = f
        # self.redg residual edge to be added inside instance method
        
    def __repr__(self):
        return "%s --> %s : cap %s flow %s" % (self.source, self.sink, self.capacity, self.flow)
        
class Graph(object):
    def __init__(self):
        self.vset = {}
        
    def __repr__(self):
        return "graph as %s vertices" % (len(self.vset))
        
    def add_vertex(self, vertex):
        self.vset[vertex] = []
        
    def get_edges(self, v):
        return self.vset[v]
        
    def add_edge(self, u, v, w=0):
        if (u == v):
            raise ValueError("u == v")
        edge = Edge(u, v, w, 0)
        redge = Edge(v, u, 0, 0)
        edge.redge = redge
        redge.redge = edge
        self.vset[u].append(edge)
        self.vset[v].append(redge)
    
    def reset_flow(self):
        for v in self.vset:
            for edge in self.vset[v]:
                edge.flow = 0
                
    def report_flow(self):
        for v in self.vset:
            for edge in self.vset[v]:
                print ("%s --> %s flow %s" % (edge.source, edge.sink, edge.flow))
        
    def find_path(self, source, sink, path):
        if (source == sink):
            return path
        for edge in self.get_edges(source):
            residual = edge.capacity - edge.flow
            if (residual > 0) and (edge not in path) and (edge.redge not in path) :
                result = self.find_path(edge.sink, sink, path + [edge])
                if result != None:
                    return result
    
    # This method is not used in assignment problem
    def max_flow(self, source, sink):
        path = self.find_path(source, sink, [])
        while path != None:
            residuals = [edge.capacity - edge.flow for edge in path]
            flow = min(residuals)
            for edge in path:
                edge.flow += flow
                edge.redge.flow -= flow
            print (flow, path, [edge.flow for edge in path], residuals)
            path = self.find_path(source, sink, [])
        return sum(edge.flow for edge in self.get_edges(source))
    
    # Find maximum cardinality matching
    def b_matching(self, source, sink):
        path = self.find_path(source, sink, [])
        while path != None:
            residuals = [edge.capacity - edge.flow for edge in path]
            flow = min(residuals)
            for edge in path:
                edge.flow += flow
                edge.redge.flow -= flow
            # print (flow, path, [edge.flow for edge in path], residuals)
            path = self.find_path(source, sink, [])
        match = [(e.sink, m.sink)
                for e in self.get_edges(source) if e.flow == 1
                for m in self.get_edges(e.sink) if m.flow == 1]      
        return match
    
    # Find minimum set of vertices that cover all edges
    # This version is inefficient (Up to 50X50 Graph is solvable
    # More refinement is needed
    def v_cover(self, source, sink, match):
        L = {e.sink for e in self.get_edges(source)}
        R = {e.sink for e in self.get_edges(sink)}
        ML = {t[0] for t in match}
        MR = {t[1] for t in match}
        U = L - ML
        Z = set()
        Q = list(U)
        while Q:
            q = Q.pop()
            if q not in Z:
                Z.add(q)
                if q in L:
                    for e in self.vset[q]:
                        if e.sink not in Z and e.sink in R and e.flow != 1:
                            Q.append(e.sink)
                elif q in R:
                    for e in self.vset[q]:
                        if e.sink not in Z and e.sink in ML and e.redge.flow == 1:
                            Q.append(e.sink)
        K = (L - Z).union(R.intersection(Z))
        return K
 

In [202]:
'''This cell find a minimum weight matching of a 
   weighted complete bipartite graph which model an assignment problem
''' 

# Find a minimum set of lines that coverr all zeros in Matrix a
def min_cover(a):
    # Construct a bipartite graph representing Matrix a
    (Nh, Nw) = a.shape
    h = Graph()
    h.add_vertex('s')                  # Add Source vertex 's'
    h.add_vertex('t')                  # Add Sink vertex 't'
    for i in range(Nh):                # Add Left partite vetices, one for each row
        h.add_vertex('L'+str(i))
        h.add_edge('s', 'L'+str(i), 1)
    for j in range(Nw):                # Add Right partite vertices, one for each column
        h.add_vertex('R'+str(j))
        h.add_edge('R'+str(j), 't', 1)
    for i in range(Nh):                # Add edges between Left and Right Partites, one for each zero
        for j in range(Nw):
            if a[i, j] == 0:
                h.add_edge('L'+str(i), 'R'+str(j), 1)
                
    h.reset_flow()                  # Reset all flow to zero
    match = h.b_matching('s', 't')  # Find a maximum cardinality match
    cover = h.v_cover('s', 't', match)  # Find a minimum set of vertices covering all edges
    
    print ("Cost Matrix\n", a)
    print ("Matching Result\n", len(match), match)
    print ("Min Vertex Cover\n", len(cover), cover)
    return cover, match

# Assignment problem as Minimum Weighted Matching of a complete bipartite graph
# The assignment problem is modeled as a square matrix a
def matching(a):
    (Nh, Nw) = a.shape
    if Nh != Nw or Nh < 1 or Nw < 1 or np.min(a) < 0:
        raise ValueError("Incorrect Matrix")
    for i in range(Nh):            # Row reduction
        a[i, :] -= np.min(a[i, :])
    for j in range(Nw):            # Column reduction
        a[:, j] -= np.min(a[:, j])        
    while True:
        print ("\n\nA New Iteration")
        cover, match = min_cover(a)       # Find minimum set of lines covering all zeros
        if len(cover) == Nh:       # Optimal assignment found if 
            return cover, match, a
        else:
            h_covered = [int(c[1:]) for c in cover if c[0] == 'L']
            v_covered = [int(c[1:]) for c in cover if c[0] == 'R']
            h_uncovered = list(set(i for i in range(Nh)) - set(h_covered))
            v_uncovered = list(set(j for j in range(Nw)) - set(v_covered))
            min_uncovered = min([a[i, j] for i in h_uncovered for j in v_uncovered])
            for i in h_uncovered:
                    a[i, :] -= min_uncovered
            for j in v_covered:
                    a[:, j] += min_uncovered

In [203]:
a = np.array([[50, 100, 20], [30, 80, 70], [75, 65, 35]])

print ("Initial Matrix\n", a)
cover, match, sol_matrix = matching(a)
print ("\nFound Assignment")
print ("Cost Matrix\n", sol_matrix)
print ("Matching Result\n", len(match), match)
print ("Min Vertex Cover\n", len(cover), cover)


Initial Matrix
 [[ 50 100  20]
 [ 30  80  70]
 [ 75  65  35]]


A New Iteration
Cost Matrix
 [[30 50  0]
 [ 0 20 40]
 [40  0  0]]
Matching Result
 3 [('L0', 'R2'), ('L1', 'R0'), ('L2', 'R1')]
Min Vertex Cover
 3 {'L0', 'L1', 'L2'}

Found Assignment
Cost Matrix
 [[30 50  0]
 [ 0 20 40]
 [40  0  0]]
Matching Result
 3 [('L0', 'R2'), ('L1', 'R0'), ('L2', 'R1')]
Min Vertex Cover
 3 {'L0', 'L1', 'L2'}


In [204]:
b = np.array([[90, 75, 75, 80],
             [35, 85, 55, 65],
             [125, 95, 90, 105],
             [45, 110, 95, 115]])

print ("Initial Matrix\n", b)
cover, match, sol_matrix = matching(b)
print ("\nFound Assignment")
print ("Cost Matrix\n", sol_matrix)
print ("Matching Result\n", len(match), match)
print ("Min Vertex Cover\n", len(cover), cover)

Initial Matrix
 [[ 90  75  75  80]
 [ 35  85  55  65]
 [125  95  90 105]
 [ 45 110  95 115]]


A New Iteration
Cost Matrix
 [[15  0  0  0]
 [ 0 50 20 25]
 [35  5  0 10]
 [ 0 65 50 65]]
Matching Result
 3 [('L0', 'R1'), ('L1', 'R0'), ('L2', 'R2')]
Min Vertex Cover
 3 {'R0', 'L0', 'L2'}


A New Iteration
Cost Matrix
 [[35  0  0  0]
 [ 0 30  0  5]
 [55  5  0 10]
 [ 0 45 30 45]]
Matching Result
 3 [('L0', 'R1'), ('L1', 'R0'), ('L2', 'R2')]
Min Vertex Cover
 3 {'R2', 'R0', 'L0'}


A New Iteration
Cost Matrix
 [[40  0  5  0]
 [ 0 25  0  0]
 [55  0  0  5]
 [ 0 40 30 40]]
Matching Result
 4 [('L0', 'R1'), ('L1', 'R3'), ('L2', 'R2'), ('L3', 'R0')]
Min Vertex Cover
 4 {'L3', 'L1', 'L0', 'L2'}

Found Assignment
Cost Matrix
 [[40  0  5  0]
 [ 0 25  0  0]
 [55  0  0  5]
 [ 0 40 30 40]]
Matching Result
 4 [('L0', 'R1'), ('L1', 'R3'), ('L2', 'R2'), ('L3', 'R0')]
Min Vertex Cover
 4 {'L3', 'L1', 'L0', 'L2'}


In [205]:
c = np.random.randint(30, 300, size=(30, 30))  # Limit the size to (50, 50) for reasonable run time

print ("Initial Matrix\n", c)
cover, match, sol_matrix = matching(c)
print ("\nFound Assignment")
print ("Cost Matrix\n", sol_matrix)
print ("Matching Result\n", len(match), match)
print ("Min Vertex Cover\n", len(cover), cover)

Initial Matrix
 [[226 170 134 275 237  45 153  38 244 145 270  75  48 107 200 141 243 176
  222 285 137 115 178 178 255  99 206 199 163 218]
 [285 121  35 100 247 276 158 108 248 180 257 223 171 228 121 158 105 217
  172  78  67 223 141 185 198  99  98 214 297 146]
 [230  32 111  81  99 122 150  79 218 238 191 193 236  59 121 299 243 287
  107  83  30  53  71  30 150 242 247 283  45 252]
 [202 253 139 295 266 204 256  53  78 113 222 118  42 180 143 251  62 287
  254 128 166  92 272 201 211 216  63 288 249 266]
 [ 83 267 109 266 188 101 254 147 259 253 288 141 175 152  63 204 164 259
  296 209 195  54 158  66 261  35  34 264 164 258]
 [286 297  95  42  34 242 150 198 127  53 275 163 161  99 206 143  91 175
  265 113 196 237  32 189 193 176 200 214 168 226]
 [159 206 179  73 233  93 280 174 237 148  75 127  30  77  44 203 286 231
  116 190 240 125 193 115  63 123 187  87  46 233]
 [175  74  90 110 268 128 203  90  97  47 299 176  97  32 212 265  49 129
  192  36 134 238 218 148 244 133 2